In [0]:
import tensorflow as tf, pandas as pd, numpy as np, time, os

In [2]:
!pip install pydub

In [0]:
# pydub
from pydub import AudioSegment

In [4]:
!git clone https://github.com/RedbirdTaiwan/silic.git

Cloning into 'silic'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 48 (delta 8), reused 29 (delta 7), pack-reused 18
Unpacking objects: 100% (48/48), done.


In [5]:
!ls silic/
!ls silic/model
!ls silic/sample

LICENSE  model	README.md  sample  scripts
conv_labels.txt  frozen_graph.pb  sound_class.txt
11794.mp3


In [0]:
class Autolabel():
  def __init__(self, label_file, graph_file):
    self.label_file = label_file
    self.graph_file = graph_file
    self.__sound = ''
    self.results = {}

  def readfile(self, audio_file):
    frame_rate = 30000
    filext = audio_file[-3:].lower()
    if filext == "mp3":
        self.__sound = AudioSegment.from_mp3(audio_file)
    elif filext == "wma":
        self.__sound = AudioSegment.from_file(audio_file, "wma")
    elif filext == "m4a":
        self.__sound = AudioSegment.from_file(audio_file, "m4a")
    elif filext == "ogg":
        self.__sound = AudioSegment.from_ogg(audio_file)
    else:
        self.__sound = AudioSegment.from_wav(audio_file)
    if self.__sound.frame_rate > frame_rate:
        self.__sound = self.__sound.set_frame_rate(frame_rate)
    if self.__sound.channels > 1:
        self.__sound = self.__sound.split_to_mono()[0]
    if not self.__sound.sample_width == 2:
        self.__sound = self.__sound.set_sample_width(2)

  def ailabel(self, **kwargs):
    if not len(self.__sound):
      print('use ".readfile(<audio_file_path>)" to import a recording')
      return false
    sample_rate = 30000
    clip_duration_ms = 1000
    if not 'step' in kwargs.keys():
      step = 1000
    else:
      step = kwargs['step']
    labels_list = [line.rstrip() for line in tf.io.gfile.GFile(self.label_file)]
    with tf.io.gfile.GFile(self.graph_file, 'rb') as f:
      graph_def = tf.compat.v1.GraphDef()
      graph_def.ParseFromString(f.read())
      tf.import_graph_def(graph_def, name='')
    sess = tf.compat.v1.Session()
    softmax_tensor = sess.graph.get_tensor_by_name('labels_softmax:0')
    self.results = {k:[] for k in labels_list[2:]}
    for t in range(0,len(self.__sound),step):
      if (t + clip_duration_ms) < len(self.__sound):
        soundclip = self.__sound[t:t+clip_duration_ms]
        soundclip.export('tmp.wav', format="wav")
        with open('tmp.wav', 'rb') as wav:
          wav_data2 = wav.read()
        predictions, = sess.run(softmax_tensor, {'wav_data:0': wav_data2})
        for node_id in range(2, len(labels_list)):
          song_of_bird = labels_list[node_id]
          score = round(predictions[node_id],3)
          self.results[song_of_bird].append([t,score])

In [0]:
test = Autolabel('silic/model/conv_labels.txt', 'silic/model/frozen_graph.pb')

In [0]:
test.readfile('silic/sample/11794.mp3')

In [0]:
test.ailabel(step=250)

In [19]:
results = test.results
soundclass = pd.read_csv('silic/model/sound_class.txt', index_col=0, sep='\t').T.to_dict()
for soundid, scores in zip(results.keys(), results.values()):
  for score in scores:
    if score[1] >= 0.8:
      sound = soundclass[int(soundid)]['scientific_name'] + ':' + soundclass[int(soundid)]['class_name']
      time_start = score[0]/1000
      ai_score = score[1]
      print(sound, time_start, ai_score)

Otus spilocephalus:S-01 0.75 0.834
Otus spilocephalus:S-01 35.25 0.835
Otus spilocephalus:S-01 163.75 0.868
Otus spilocephalus:S-01 219.25 0.826
Otus spilocephalus:S-01 227.75 0.922
Otus spilocephalus:S-01 233.0 0.844
Otus spilocephalus:S-01 246.75 0.846
Otus spilocephalus:S-01 247.75 0.936
Otus spilocephalus:S-01 252.75 0.805
Otus spilocephalus:S-01 253.5 0.905
Otus spilocephalus:S-01 260.5 0.819
Ninox japonica:S-01 3.0 0.827
Ninox japonica:S-01 5.75 0.828
Ninox japonica:S-01 37.5 0.951
Ninox japonica:S-01 38.0 0.809
Ninox japonica:S-01 38.75 0.909
Ninox japonica:S-01 40.0 0.981
Ninox japonica:S-01 40.75 0.858
Ninox japonica:S-01 41.25 0.938
Ninox japonica:S-01 42.0 0.811
Ninox japonica:S-01 43.25 0.92
Ninox japonica:S-01 43.75 0.902
Ninox japonica:S-01 44.0 0.946
Ninox japonica:S-01 44.5 0.9
Ninox japonica:S-01 45.25 0.902
Ninox japonica:S-01 46.5 0.965
Ninox japonica:S-01 47.25 0.817
Ninox japonica:S-01 47.75 0.9
Ninox japonica:S-01 50.5 0.95
Ninox japonica:S-01 51.75 0.948
Ninox ja